In [2]:
import pandas as pd
from datetime import datetime
import requests
import numpy_financial as npf
import numpy as np
import math
import matplotlib.pyplot as plt
import json

In [3]:
data = pd.read_csv("./ОФЗ Анализ - Погашение после 2014 года.csv")
display(data)

,Название,Начало Торгов,Погашение,Купон,Периодичность,ISIN
0,25075,13.02.2012,15.07.2015,"6,88",2,SU25075RMFS1
1,25077,13.02.2012,20.01.2016,7.35,2,SU25077RMFS7
2,25079,13.02.2012,03.06.2015,7,2,SU25079RMFS3
3,25080,25.04.2012,19.04.2017,7.4,2,SU25080RMFS3
4,25081,06.02.2013,31.01.2018,6.2,2,SU25081RMFS4
5,25082,19.06.2013,11.05.2016,6,2,SU25082RMFS4
6,25083,18.10.2017,15.12.2021,7,2,SU25083RMFS5
7,25084,19.02.2020,04.10.2023,5.3,2,SU25084RMFS5
8,25085,22.09.2021,24.09.2025,6.4,2,SU25085RMFS6
9,26202,13.02.2012,17.12.2014,11.2,4,SU26202RMFS0


In [4]:
def numberOfDays(date1, date2, format="%d.%m.%Y"):
  return float((datetime.strptime(date1, format) - datetime.strptime(date2, format)).days)

In [5]:
# Adding date columns
data["start"] = data.apply(lambda row: datetime.strptime(row["Начало Торгов"], "%d.%m.%Y"), axis=1)
data["end"] = data.apply(lambda row: datetime.strptime(row["Погашение"], "%d.%m.%Y"), axis=1)

In [160]:
# Pick variables
cur_date_str = "01.01.2025"
rate = 13
inflation = 6
# Process
cur_date = datetime.strptime(cur_date_str, "%d.%m.%Y")
print(cur_date)
# Select papers
cur_data = data[(data["start"] < cur_date) & (data["end"] > cur_date)]
cur_data.reset_index(drop=True, inplace=True)

2025-01-01 00:00:00


In [161]:
# Calculate yield to maturity
cur_data["days_to_maturity"] = cur_data.apply(lambda row: (row["end"] - cur_date).days, axis=1)
# Convert column data types
cur_data["Купон"] = pd.to_numeric(cur_data["Купон"])
cur_data["Периодичность"] = pd.to_numeric(cur_data["Периодичность"])
sorted_result = cur_data.sort_values("days_to_maturity")
sorted_result["years_to_maturity"] = sorted_result["days_to_maturity"] / 365
sorted_result = sorted_result[sorted_result["days_to_maturity"] >= 365/2]

C:\Users\igor\AppData\Local\Temp\ipykernel_4132\3321616505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_data["days_to_maturity"] = cur_data.apply(lambda row: (row["end"] - cur_date).days, axis=1)
C:\Users\igor\AppData\Local\Temp\ipykernel_4132\3321616505.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_data["Купон"] = pd.to_numeric(cur_data["Купон"])
C:\Users\igor\AppData\Local\Temp\ipykernel_4132\3321616505.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [162]:
# Forecasting
def ytm(row, rate, inflation):
  # Model 16
  if (rate >10):
    return 9.2406 + 0.2647 * rate - 0.1544 * inflation
  # Model 15
  if ((rate > 8) & (rate <=10)):
    return 7.4417 + 0.3805 * rate - 0.1504 * inflation
  # Model 14
  if ((rate > 7.5) & (rate <=8)):
    return 6.7851 + 0.0695 * inflation + 0.4179 * math.log(row["years_to_maturity"])
  # Model 13
  if (rate == 7.5):
    return 7.1794 + 0.051 * inflation + 0.8118 * math.log(row["years_to_maturity"])
  # Model 11
  if ((rate >= 6) & (rate < 7.5)):
    return -0.0453 + 0.9446 * rate + 0.3014 * math.log(rate)
  # Model 12
  return -0.5088 + 1.1674 * rate + 0.7357 * math.log(row["years_to_maturity"])

In [163]:
sorted_result["ytm"] = sorted_result.apply(lambda row: ytm(row, rate, inflation), axis=1)

In [164]:
""" Shit doesnt work
moexDate = datetime.strftime(cur_date, "%Y-%m-%d")
url = "https://iss.moex.com/iss/apps/bondization/yieldscalculator?accint_source=t0&calc_method=by_yield_effective_to_maturity&calc_value=11.34&secid=SU26240RMFS0&sell_date="+ moexDate +"&sell_value=11.34&tradedate=" + moexDate
res = requests.get(url)
result = json.loads(res.text)
print(result)
price = result["calculated"]["PRICE"]
duration = result["calculated"]["DURATIONYEAR"]
print(price, duration)
"""

' Shit doesnt work\nmoexDate = datetime.strftime(cur_date, "%Y-%m-%d")\nurl = "https://iss.moex.com/iss/apps/bondization/yieldscalculator?accint_source=t0&calc_method=by_yield_effective_to_maturity&calc_value=11.34&secid=SU26240RMFS0&sell_date="+ moexDate +"&sell_value=11.34&tradedate=" + moexDate\nres = requests.get(url)\nresult = json.loads(res.text)\nprint(result)\nprice = result["calculated"]["PRICE"]\nduration = result["calculated"]["DURATIONYEAR"]\nprint(price, duration)\n'

In [165]:
""" Shit doesnt work
pv = 0
periods = 12.34
n_per = math.floor(periods)
space = np.arange(0, periods, 0.5)
for i in space:
  payment = 70/2
  pvp = payment / math.pow(1 + 0.1134, i)
  pv += pvp
delta = periods - space[-1]
last_payment = 70 * delta + 1000
print(last_payment)
pv += last_payment / math.pow(1 + 0.1134, periods)
print(pv)
"""

' Shit doesnt work\npv = 0\nperiods = 12.34\nn_per = math.floor(periods)\nspace = np.arange(0, periods, 0.5)\nfor i in space:\n  payment = 70/2\n  pvp = payment / math.pow(1 + 0.1134, i)\n  pv += pvp\ndelta = periods - space[-1]\nlast_payment = 70 * delta + 1000\nprint(last_payment)\npv += last_payment / math.pow(1 + 0.1134, periods)\nprint(pv)\n'

In [166]:
def present_value(row):
  coupon = row["Купон"]
  duration = row["years_to_maturity"]
  frequency = row["Периодичность"]
  coupon_value = 1000 / 100 * coupon / frequency
  number_of_periods = duration * frequency
  rate_of_return = math.pow(1 + row["ytm"]/100, 1 / frequency)
  cumulative_coupon_value = coupon_value * (1 - math.pow(rate_of_return, -number_of_periods))/ (rate_of_return -1)
  final_discounted_value = 1000 / math.pow(rate_of_return, number_of_periods)
  present_value = cumulative_coupon_value + final_discounted_value
  return present_value

In [167]:
sorted_result["pv"] = sorted_result.apply(present_value, axis=1)
sorted_result["Price"] = sorted_result["pv"] / 10

In [168]:
final_result = sorted_result[["Название", "Купон", "ISIN", "years_to_maturity", "ytm", "Price"]]
display(final_result)

,Название,Купон,ISIN,years_to_maturity,ytm,Price
15,26234,4.50,SU26234RMFS3,0.536986,11.7553,96.487628
0,25085,6.40,SU25085RMFS6,0.728767,11.7553,96.576603
10,26229,7.15,SU26229RMFS3,0.863014,11.7553,96.575864
4,26219,7.75,SU26219RMFS4,1.706849,11.7553,94.437995
8,26226,7.95,SU26226RMFS9,1.764384,11.7553,94.579888
1,26207,8.15,SU26207RMFS9,2.090411,11.7553,94.052363
13,26232,6.00,SU26232RMFS7,2.761644,11.7553,87.445512
2,26212,7.05,SU26212RMFS9,3.049315,11.7553,88.986777
17,26236,5.70,SU26236RMFS8,3.375342,11.7553,84.320278
18,26237,6.70,SU26237RMFS6,4.200000,11.7553,84.567144
